## Vorbereitung

2. Zelle für Generierungen mit dem spezialisierten Modell

3. Für Generierungen mit dem originalen Modell


In [ ]:
# Install packages
try:
  import gpt_2_simple as gpt2
except:
  %tensorflow_version 1.x
  !pip install -q gpt-2-simple

import os
import gpt_2_simple as gpt2


# Mount Drive
if not os.path.isdir("/content/drive/"):
  gpt2.mount_gdrive()

Loading pretrained model models/1558M/model.ckpt
INFO:tensorflow:Restoring parameters from models/1558M/model.ckpt


In [ ]:
# Load lyrics Model
checkpoint_dir='/content/drive/MyDrive/Training-Data/gpt-2/saved/1558M-41343'
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess,
              checkpoint='latest',
              run_name='1558M-41343',
              checkpoint_dir='/content/drive/MyDrive/Training-Data/gpt-2/saved/',
              model_name=None,)

In [ ]:
# Load regular Model
sess = gpt2.start_tf_sess()
gpt2.download_gpt2(model_name="1558M")
gpt2.load_gpt2(sess,model_name="1558M")

## Kapitel 4. Liedtexte generieren

Musiktexte erzeugen (ohne Eingabekontext)

In [ ]:
#1h 4min for n=100
n = 300
file_dir= "/content/drive/MyDrive/Training-Data/generated/"
file_name = file_dir + str(len(os.listdir(file_dir))) + "-generated-"+ str(n) +".txt"

gpt2.generate_to_file(
              sess,
              run_name='1558M-41343',
              checkpoint_dir='/content/drive/MyDrive/Training-Data/gpt-2/saved/',
              length=1023,
              nsamples=n,
              top_k=40,
              sample_delim="=" * 20 + "\n",
              destination_path=file_name,
              prefix="<|startoftext|>",
              truncate='<|endoftext|>',
              include_prefix=False
              )

Texte mit regulärem GPT-2 erzeugen

In [ ]:
n = 300
file_dir= "/content/drive/MyDrive/Training-Data/generated_orig/"
file_name = file_dir + str(len(os.listdir(file_dir))) + "-generated-"+ str(n) +".txt"

text = gpt2.generate(
              sess,
              length=1023,
              nsamples=n,
              destination_path=file_name,
              model_name="1558M",
              top_k=40,
              sample_delim="=" * 20 + "\n",
              truncate='<|endoftext|>',
              include_prefix=False
              )

## Kapitel 5: Cover erzeugen

In [ ]:
from tqdm import tqdm 
import pandas as pd
import numpy as np

# Input data for which to genrate covers
df = pd.read_csv("/content/drive/MyDrive/Training-Data/covers/eval_results_org.csv", index_col=0)


# Loop over every song
for index in tqdm(df.index, unit="covers"):
  try:
    if not pd.isna(df["completions"][index]):
      continue
  except KeyError :
    pass

  # get first and second half for current song
  text = df["original"][index]
  c_index = text.index("\n", int(len(text) / 2), len(text)-1)
  end = text[c_index:len(text)-1]
  start = text[0:c_index]

  # Generate 15 cover passages 
  completions = gpt2.generate(
      sess,
                run_name='1558M-41343',
                checkpoint_dir='/content/drive/MyDrive/Training-Data/gpt-2/saved/',
                return_as_list=True,
                length=1023,
                nsamples=15,
                temperature=0.7,
                prefix="<|startoftext|>" + start,
                truncate='<|endoftext|>',
                include_prefix=False
                )
  
  df["start"][index]=start
  df["end"][index]=end
  # store cover versions after every generation
  df["completions"][index]=completions
  df.to_csv("/content/drive/MyDrive/Training-Data/covers/eval_results_org.csv")



100%|██████████| 100/100 [09:00<00:00,  5.40s/covers]


## Kapitel 3 Evaluierung

Eine vervollständigung pro Lied 

In [ ]:
import pandas as pd
from tqdm import tqdm

df = pd.read_csv("/content/drive/MyDrive/Training-Data/completion-1.0.csv", index_col=0)

for index in tqdm(df.index, unit="covers"):
  if not pd.isna(df["completion"][index]):
    continue
  input = df["original"][index]
  try:
    c_index = input.index("\n", int(len(input) / 2), len(input)-1)
  except:
    continue
  end = input[c_index:len(input)-1]
  input = input[0:c_index]

  text = gpt2.generate(
                sess,
                run_name='1558M-41343',
                checkpoint_dir='/content/drive/MyDrive/Training-Data/gpt-2/saved/',
                return_as_list=True,
                length=1023,
                nsamples=1,
                top_k=40,
                prefix="<|startoftext|>" + context,
                truncate='<|endoftext|>',
                include_prefix=False
                )

  df["start"][index]=input
  df["end"][index]=end
  df["completion"][index]=text
  df.to_csv("/content/drive/MyDrive/Training-Data/completion-1.0.csv")

  7%|▋         | 297/4443 [14:54:24<806:16:55, 700.10s/covers]